## 中文文本分类

#### 文本数据特征工程

* BOW/词袋模型 CountVector
* TF-IDF TfidfVectorizer
* word2vec

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import word2vec

#### 文本分类模型

* NB/SVM/GBDT
* Fasttext
* CNN/LSTM

## 朴素贝叶斯

我们试用朴素贝叶斯完成一个中文文本分类器，一般在数据量足够，数据丰富足够的情况下，用朴素贝叶斯完成这个任务，准确度还是很不错的。

机器学习的算法要取得好效果，离不开数据，先来加载数据...

#### 准备数据

我们先挑选科技、汽车、娱乐、军事、运动共5类文本数据进行处理。

In [2]:
import jieba
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

In [4]:
technology = df_technology.content.values.tolist()[1000:21000]
technology = [x for x in [" ".join(technology[i].split()) for i in range(len(technology))] if x]
car = df_car.content.values.tolist()[1000:21000]
car = [x for x in [" ".join(car[i].split()) for i in range(len(car))] if x]
entertainment = df_entertainment.content.values.tolist()[:20000]
entertainment = [x for x in [" ". join(entertainment[i].split()) for i in range(len(entertainment))] if x]
military = df_military.content.values.tolist()[:20000]
military = [x for x in [" ".join(military[i].split()) for i in range(len(military))] if x]
sports = df_sports.content.values.tolist()[:20000]
sports = [x for x in [" ".join(sports[i].split()) for i in range(len(sports))] if x]

In [5]:
technology[12]

'现在家里都拉了网线，都能无线上网，一定要帮他们先登上WiFi，另外，老人不懂得流量是什么，也不知道如何开关，控制流量，所以设置好流量上限很重要，免得不小心点开了视频或者下载，电话费就大发了。'

In [6]:
car[100]

'截至发稿时，人人车给出的处理方案仍旧是检修车辆。王先生则认为，车辆在购买时就存在问题，但交易平台并未能检测出来。因此，王先生希望对方退款。王先生称，他将找专业机构对车辆进行鉴定，并通过法律途径维护自己的权益。J256'

In [7]:
military[10]

'央视记者 胡善敏：我现在所处的位置是在辽宁舰的飞行甲板，执行跨海区训练和试验任务的辽宁舰官兵，正在展开多个科目的训练，穿着不同颜色服装的官兵在紧张的对舰载机进行转运。'

In [8]:
sports[10]

'据统计，2016年仅在中国田径协会注册的马拉松赛事便达到了328场，继续呈现出爆发式增长的态势，2015年，这个数字还仅仅停留在134场。如果算上未在中国田协注册的纯“民间”赛事，国内全年的路跑赛事还要更多。'

### 分词与中文文本处理

#### 中文停止词过滤与生成训练数据（降噪）

In [9]:
stopword = pd.read_csv('./data/stopwords.txt', index_col=False, 
                       quoting=3, sep='\t', names=['stopword'], encoding='utf-8')
stopword = stopword.stopword.values

In [10]:
stopword

array(['!', '"', '#', ..., '07', '08', '09'], dtype=object)

In [11]:
sentences = []
def preprocess_text(content, sentences, category):
        for line in content:
            try:
                segs = jieba.lcut(line)
                segs = filter(lambda x: len(x) > 1, segs)
                segs = filter(lambda x: x not in stopword, segs)
                sentences.append((" ".join(segs), category))
            except Exception:
                print(line)
                continue
            
preprocess_text(technology, sentences, 'technology')    
preprocess_text(car, sentences, 'car') 
preprocess_text(entertainment, sentences, 'entertainment')    
preprocess_text(military, sentences, 'military')
preprocess_text(sports, sentences, 'sports')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/dq/s5dx7lqd1nl0kf3829swj_3w0000gn/T/jieba.cache
Loading model cost 1.123 seconds.
Prefix dict has been built succesfully.


#### 生成训练集

打乱顺序，生成更可靠的训练集

In [12]:
import random
random.shuffle(sentences)

In [13]:
for sentence in sentences[:10]:
    print(sentence[0], sentence[1])

本次 赛事 骑师 阵营 中将 五名 中国 优秀 骑师 中国 骑师 首次 征战 成都 迪拜 国际 五名 中国 骑师 出战 两场 骑师 挑战赛 外国 骑师 同场竞技 一较高下 刷新 中国 速度 赛马 历史 sports
本次 峰会 联盟 功能 联合 各大 正品 汽车 电子 品牌 苏宁 易购 双线 渠道 用户 提供 货真价实 产品 联盟 产品 价格 品质 售后 做出 规格 确保 苏宁 商家 联合 作保 质量 服务 双翼 齐飞 car
亮剑 行业 树立 规范 标准 technology
迄今为止 通信 行业 5G 规范 定义 以往 通信 技术 更新 一代 周期 推算 5G 2021 商用 technology
昨天下午 圆通 工作人员 媒体 回应 北京 圆通 并未 倒闭 网友 站点 正好 一辆 快递 卸车 积压 欠薪 倒闭 大面积 情况 属实 春节 期间 压货 网点 人手 导致 派件 延期 快递 延迟 派送 拨打 客服 电话 提供 投诉 投诉 成功 后会 快递 给予 一百元 处罚 technology
考生 考生 家长 entertainment
当天 影片 众生 预告片 首度 曝光 唐僧 师徒 西天 取经 一路 斩妖 降魔 吴亦凡 饰演 唐僧 为情所困 导致 团队 内部 裂痕 矛盾重重 更新 饰演 孙悟空 扬言 师父 恩断义绝 摘掉 头上 金箍 唐僧 entertainment
中国 汽车 合资企业 走过 三十余年 资本 合作 技术 合作 品牌 技术 合作 深度 阶段 广汽 三菱 发布 全新 企业 标识 传达 出广汽 集团 三菱 汽车 三菱 商事 平等 互利 发展 态势 广汽 集团 三菱 汽车 紧密 合作 深化 沟通 合作 机制 短短 一个月 高层 交流 会晤 步入 合作 深化 阶段 car
中国 电子商务 发展 南亚 东南亚 国家 方兴未艾 世界 各大 电商 巨头 看好 潜力 无限 东南亚 南亚 市场 technology
每座 城市 年轻人 忙于 生活 初入 社会 满怀 憧憬 日子 趋于 平淡 想要 重拾 过往 轻松 步伐 难上加难 平淡 转为 有趣 知名 主持人 百克 秘诀 工作 空隙 城市 展开 有趣 探索 周五 常德路 澳门路 新疆 巴扎 土特产 新疆 老乡 上海 改良 新疆 菜馆 品尝 美食 小店 充当 军师 空闲 时间 来到 黄埔江 唯一 封闭式 内陆 复兴

使用sklearn分割训练集数据，得到部分验证集数据用于检测分类器效果

In [14]:
from sklearn.model_selection import train_test_split

X, y = zip(*sentences)
X, y = zip(*[(x, y) for x, y in zip(X, y) if x !=''])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

基于词袋模型的文本特征抽取

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

CountVec = CountVectorizer(
    analyzer='word',
    ngram_range=(1, 4),
    max_features=20000)
CountVec.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

这里可以通过调整ngram_range来扩大词向量的个数，也就是增大特征输入，默认是1_gram

#### 训练模型

In [16]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(CountVec.transform(X_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

看一下准确率如何

In [17]:
classifier.score(CountVec.transform(X_test), y_test)

0.8777415053565681

### 交叉验证

更可靠的验证方式是交叉验证，但是交叉验证最好也需要保证样本类别的均衡性，在这里我们使用StratifiedKFold

In [18]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

scores = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_index, test_index in skf.split(CountVec.transform(X), np.array(y)):
    X_train, X_test = CountVec.transform(X)[train_index], CountVec.transform(X)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    scores.append(mnb.score(X_test, y_test))

In [19]:
np.mean(scores)

0.8828066442099336

### 定义为Class类对象

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

class TextClassifier():
    
    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), 
                                          max_features=20000)
        
    def features(self, X):
        return self.vectorizer.transform(X)
    
    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)
        
    def predict(self, x):
        return self.classifier.predict(self.features([x]))
    
    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [22]:
text_classifier = TextClassifier()
text_classifier.fit(X_train, y_train)

In [23]:
text_classifier.predict('苹果 公司 有 新 的 发布 计划')

array(['technology'], dtype='<U13')

In [24]:
text_classifier.score(X_test, y_test)

0.8777415053565681

### SVM文本分类

In [25]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(CountVec.transform(X_train), y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
svc.score(CountVec.transform(X_test), y_test)

0.8490045519334223

### 写成Class

In [27]:
class TextClassifier():
    
    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [28]:
text_classifier = TextClassifier()
text_classifier.fit(X_train, y_train)

In [29]:
text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习')

array(['military'], dtype='<U13')

In [30]:
text_classifier.score(X_test, y_test)

0.8814198353947308